# 回归

## 线性回归

### 解析解

模型：$\hat{y}(\boldsymbol{w}, \boldsymbol{x}) = w_0 + w_1 x_1 + ... + w_p x_p = \boldsymbol{x}^T\boldsymbol{w}, \boldsymbol{x} = (1,x_1,x_2,\dotsc,x_p)^T$

对于数据矩阵$\boldsymbol{X} = (\boldsymbol{x}_1,\boldsymbol{x}_2,\dotsc,\boldsymbol{x}_n)^T = (1,\boldsymbol{x}^{(1)},\dotsc, \boldsymbol{x}^{(p)})$而言，取损失函数为均方误差，则我们要优化：
$$\min_{\boldsymbol{w}} \frac{1}{2n}|| \boldsymbol{X} \boldsymbol{w} - \boldsymbol{y}||_2^2$$

当$X$可逆时，可以求出解析解$\boldsymbol{w}_{opt} = (\boldsymbol{X}^T\boldsymbol{X})^{-1}\boldsymbol{X}^T\boldsymbol{Y}$.

另外，也可以用优化算法求解。

In [15]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [16]:
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)
diabetes_X.shape, diabetes_y.shape

((442, 10), (442,))

In [17]:
np.random.seed(520)
# 划分训练集与测试集
index1 = np.random.choice(diabetes_X.shape[0],round(diabetes_X.shape[0]*0.7),replace = False)
index2 = np.array(list(set(range(diabetes_X.shape[0]))-set(index1)))
X_train, X_test, y_train, y_test = (diabetes_X[index1,].copy(), diabetes_X[index2,].copy(), 
                                    diabetes_y[index1].copy(), diabetes_y[index2].copy())
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((309, 10), (309,), (133, 10), (133,))

In [18]:
# 直接求出的解析解
X = np.concatenate((np.ones((X_train.shape[0],1)),X_train),axis = 1) # 增加截距项对应的列
w_analy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y_train)
w_analy

array([ 1.52637181e+02, -2.46209538e-01, -2.95443035e+02,  4.76099495e+02,
        3.97871650e+02, -1.88230740e+02, -9.99115610e+01, -1.60960325e+02,
        2.62672665e+02,  3.71770233e+02,  1.23301671e+02])

In [19]:
# 定义损失函数
def half_mean_squared_error(y,yhat):
    return np.mean((y-yhat)**2)/2

In [20]:
X_test_with_intercept = np.concatenate((np.ones((X_test.shape[0],1)),X_test),axis = 1)
ypred = np.dot(X_test_with_intercept, w_analy)
half_mean_squared_error(ypred,y_test)

1762.5796693482537

### 优化算法求解

#### 梯度下降法

In [21]:
# 梯度下降法
class GD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, params, grads):
        params -= self.lr * grads
# 计算
w = np.zeros(X.shape[1])
l = [None]*50002
l[0] = 0
optimizer = GD(lr = 0.05)
n = X.shape[0]
for i in range(50001):
    grads = X.T.dot(X.dot(w)-y_train)/n
    optimizer.update(w,grads)
    l[i+1] = half_mean_squared_error(X.dot(w),y_train)
    if i % 10000 == 0:
        print(f"第{i+1}次循环，参数值为 w = {w},损失值为 {l[i+1]}")
    if abs(l[i+1]-l[i])<1e-5:
        print(f"第{i+1}次循环，参数值为 w = {w},损失值为 {l[i+1]}")
        break

第1次循环，参数值为 w = [ 7.58058252e+00  2.81804294e-02  6.81555645e-03  9.23709419e-02
  7.35696834e-02  3.30307972e-03  5.89569285e-03 -8.34563226e-02
  6.45334982e-02  8.49922060e-02  5.74607017e-02],损失值为 13168.83063717651
第10001次循环，参数值为 w = [ 152.83924579   34.02895437 -127.05256085  381.18960598  293.10521909
  -30.30104701  -63.53814577 -218.70391321  152.53871819  282.20840365
  157.61287028],损失值为 1393.8152006429543
第20001次循环，参数值为 w = [ 152.70195529   11.25607372 -222.1676383   453.53888563  350.162037
  -68.04826278 -118.73283879 -239.21888505  155.19111715  328.86012402
  149.63918086],损失值为 1338.5315265229856
第30001次循环，参数值为 w = [ 152.66047046    3.19613619 -262.62486891  472.36328819  372.11207184
  -79.0192716  -136.49237837 -243.49794476  162.79143672  345.56446104
  139.8659575 ],损失值为 1331.1553504982137
第40001次循环，参数值为 w = [ 152.64698248    0.59383743 -279.62865081  476.20120728  382.33153341
  -83.26181755 -143.33758053 -243.53332089  171.10699209  351.44925921
  133.38427982],损失值为

下降得非常慢，需要其他加速的方法。

#### Adagrad算法

In [22]:
# Adagrad算法
class Adagrad:
    def __init__(self, lr):
        self.lr = lr
        self.h = None
    def update(self,params, grads):
        if self.h is None:
            self.h = np.zeros_like(grads)
        self.h = grads * grads
        params -= self.lr * grads/(np.sqrt(self.h)+1e-7)

In [23]:
# 计算
w = np.zeros(X.shape[1])
l = [None]*10002
l[0] = 0
optimizer = Adagrad(lr = 0.1)
n = X.shape[0]
for i in range(10001):
    grads = X.T.dot(X.dot(w)-y_train)/n
    optimizer.update(w,grads)
    l[i+1] = half_mean_squared_error(X.dot(w),y_train)
    if i % 3000 == 0:
        print(f"第{i+1}次循环，参数值为 w = {w},损失值为 {l[i+1]}")
    if abs(l[i+1]-l[i])<1e-5:
        print(f"第{i+1}次循环，参数值为 w = {w},损失值为 {l[i+1]}")
        break

第1次循环，参数值为 w = [ 0.1         0.09999998  0.09999993  0.09999999  0.09999999  0.09999985
  0.09999992 -0.09999999  0.09999999  0.09999999  0.09999999],损失值为 14273.978897622457
第3001次循环，参数值为 w = [ 152.49976475   24.70584239 -253.49964443  300.09996473  300.09995025
   26.07494417 -224.89848863 -298.50072491  227.10053604  300.09992983
  188.70530711],损失值为 1375.83291560383
第5078次循环，参数值为 w = [ 1.52600765e+02 -2.88960995e-01 -2.95183540e+02  4.76597671e+02
  3.97587293e+02 -1.30949242e+02 -1.45689524e+02 -1.85780528e+02
  2.56322022e+02  3.51029403e+02  1.23535718e+02],损失值为 1328.561658537623


很快就收敛了。

#### Adam算法

In [24]:
# Adam算法
class Adam:
    def __init__(self, lr = 0.002, beta1 = 0.9, beta2 = 0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.m = None
        self.v = None
        self.um = None
        self.uv = None
        self.iter = 0
    def update(self, params, grads):
        if self.m is None:
            self.m = np.zeros_like(grads)
            self.v = np.zeros_like(grads)
            self.um = np.zeros_like(grads)
            self.uv = np.zeros_like(grads)
        self.iter += 1
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * grads ** 2
        self.um = self.m / (1 - self.beta1 ** self.iter)
        self.uv = self.v / (1 - self.beta2 ** self.iter)
        params -= self.lr * self.um / (np.sqrt(self.uv) + 1e-7)

In [25]:
# 计算
w = np.zeros(X.shape[1])
l = [None]*10002
l[0] = 0
optimizer = Adam(lr = 100)
n = X.shape[0]
for i in range(10001):
    grads = X.T.dot(X.dot(w)-y_train)/n
    optimizer.update(w,grads)
    l[i+1] = half_mean_squared_error(X.dot(w),y_train)
    if i % 50 == 0:
        print(f"第{i+1}次循环，参数值为 w = {w},损失值为 {l[i+1]}")
    if abs(l[i+1]-l[i])<1e-5:
        print(f"第{i+1}次循环，参数值为 w = {w},损失值为 {l[i+1]}")
        break

第1次循环，参数值为 w = [ 99.99999993  99.99998226  99.99992664  99.99999459  99.9999932
  99.99984863  99.99991519 -99.99999401  99.99999225  99.99999412
  99.9999913 ],损失值为 3424.270114090812
第51次循环，参数值为 w = [ 144.34745848   -1.63344851 -279.36689667  461.97751198  399.55927795
 -105.63342143 -160.68962894 -194.64441739  249.76698037  318.20300525
  103.9331644 ],损失值为 1365.0407525123767
第101次循环，参数值为 w = [ 153.22237653   -2.26634313 -295.2089584   476.57227615  399.0996865
 -148.8505582  -129.04881891 -177.19192634  257.39052996  357.78952428
  124.44569058],损失值为 1328.6977645223342
第151次循环，参数值为 w = [ 1.52613979e+02 -3.16814587e-01 -2.95591080e+02  4.76313174e+02
  3.97827603e+02 -1.76697619e+02 -1.09291863e+02 -1.66118047e+02
  2.61340005e+02  3.67569675e+02  1.23488612e+02],损失值为 1328.5050320441696
第179次循环，参数值为 w = [ 1.52627004e+02 -2.83394285e-01 -2.95423312e+02  4.76123390e+02
  3.97875814e+02 -1.83095385e+02 -1.03979719e+02 -1.63211065e+02
  2.62061767e+02  3.69860414e+02  1.23295711e+02],损失

收敛速度特别快。

In [32]:
# 封装为类
class LinearReg:
    def __init__(self, Y, X, optimizer = Adam, max_iter = 1000, tol = 1e-5):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.optimizer = optimizer
        self.max_iter = max_iter
        self.tol = tol
        self.center, self.scale, self.Xs, self.center_y, self.ys = self.standardized()
        self.beta = np.zeros(self.n_feature)
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        center_y = np.mean(self.Y)
        ys = self.Y - center_y
        return (center, scale, Xs, center_y, ys)
    @staticmethod
    def L2_norm(x,y):
        return np.sqrt(np.sum((x-y)**2))
    def fit(self):
        for i in range(self.max_iter):
            beta_old = self.beta.copy()
            grads = self.Xs.T.dot(self.Xs.dot(self.beta)-self.ys)/self.n
            self.optimizer.update(self.beta, grads)
            if self.L2_norm(self.beta, beta_old) < self.tol:
                self.beta = self.unstandardized()
                break
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = self.center_y - np.sum(self.beta/self.scale*self.center)
        beta[1:] = self.beta/self.scale
        return beta
    def predict(self, newdata):
        return np.dot(newdata, self.beta[1:]) + self.beta[0]

In [34]:
adam = Adam(lr=10)
linear_reg = LinearReg(Y = y_train,
                       X = X_train,
                       optimizer = adam)
linear_reg.fit()
half_mean_squared_error(linear_reg.predict(X_test),y_test)

1762.5798759097024

## 带正则项的线性回归

### lasso-L1正则

在最小二乘回归的损失函数的基础上添加 $L_1$ 正则项，即$\min_{\boldsymbol{w}} { \frac{1}{2n} ||\boldsymbol{X} \boldsymbol{w} - \boldsymbol{y}||_2 ^ 2 + \alpha ||\boldsymbol{w}||_1}$，在$X$不是列正交的时候，无法求得解析解。一般使用优化算法如坐标下降法求解。

![avatar](./images/lasso.png)

算法如下：
![avatar](./images/lasso_coordinate_descent.png)

In [13]:
#### 坐标下降法解lasso
# X 中心标准化，注意因为要进行中心标准化，所以不需要添加截距项
center = np.mean(X_train,axis = 0)
scale = np.std(X_train,axis = 0)
Xs = (X_train-center)/scale
# y 中心化
center_y = np.mean(y_train)
ys = y_train - center_y
# 定义软阈值函数
def soft_threshold(z,l):
    if z > l:
        return z - l
    elif z < - l:
        return z + l
    else:
        return 0
# 初始化参数与残差
n,p = Xs.shape
w = np.zeros(p)
r = ys.copy() # 注意深浅拷贝，当后续不会对此值进行更改时，浅拷贝没问题，但如果会修改这个值，需要深拷贝
# 设置惩罚系数
lambda_ = 0.1
# 记录损失函数变化
l = [None] * 2002
l[0] = 0
# 坐标下降
for i in range(2001):
    for j in range(p):
        z = np.dot(Xs[:,j],r)/n + w[j]
        delta = soft_threshold(z,lambda_) - w[j]
        w[j] += delta
        r -= np.dot(Xs[:,j], delta)
    w_lasso = np.zeros(X.shape[1])
    w_lasso[0] = center_y - w.dot(center/scale)
    w_lasso[1:] = w/scale
    l[i+1] = half_mean_squared_error(X.dot(w_lasso),y_train)+lambda_*np.sum(np.abs(w_lasso))
    if i % 200 == 0:
        print(f"第{i+1}次循环，参数值为 w = {w_lasso},损失值为 {l[i+1]}")
    if abs(l[i+1]-l[i])<1e-10:
        print(f"第{i+1}次循环，参数值为 w = {w_lasso},损失值为 {l[i+1]}")
        break

第1次循环，参数值为 w = [ 153.14056106  302.10730732   22.00102648  826.51095568  303.88606276
  -87.54631139  -19.92328089 -258.3364583    64.06797214  166.45880937
  -39.70688438],损失值为 1873.7173863947012
第201次循环，参数值为 w = [ 152.649404      0.         -290.91769361  475.35309925  395.41293111
 -155.91347957 -116.34019075 -178.3783239   246.68311849  362.51735282
  121.74201768],损失值为 1578.1697360447038
第401次循环，参数值为 w = [ 152.64865235    0.         -290.92566999  475.33907579  395.42106475
 -158.15236287 -114.55410203 -177.42386119  246.89874089  363.33365394
  121.74353953],损失值为 1578.2206650439953
第601次循环，参数值为 w = [ 152.64862199    0.         -290.92599222  475.33850926  395.42139333
 -158.24280983 -114.48194718 -177.38530257  246.90745165  363.36663108
  121.74360101],损失值为 1578.222726203217
第801次循环，参数值为 w = [ 152.64862076    0.         -290.92600524  475.33848638  395.4214066
 -158.24646372 -114.47903225 -177.38374487  246.90780355  363.36796329
  121.7436035 ],损失值为 1578.2228094764996
第1001次循环，

In [14]:
# 在测试集上的损失
half_mean_squared_error(X_test_with_intercept.dot(w_lasso),y_test)

1760.9303711708997

In [43]:
# 封装为类
class Lasso:
    def __init__(self, Y, X, lambda_, max_iter = 1000, tol = 1e-4):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.lambda_ = lambda_
        self.max_iter = max_iter
        self.tol = tol
        self.beta = np.zeros(self.n_feature)
        self.center, self.scale, self.Xs, self.center_y, self.ys = self.standardized()
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        center_y = np.mean(self.Y)
        ys = self.Y - center_y
        return (center, scale, Xs, center_y, ys)
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = self.center_y - np.sum(self.beta/self.scale*self.center)
        beta[1:] = self.beta/self.scale
        return beta
    @staticmethod
    def L2_norm(x):
        return np.sqrt(np.sum(x**2))
    def soft_threshold(self, z):
        if z < -self.lambda_:
            return z + self.lambda_
        elif z > self.lambda_:
            return z - self.lambda_
        else:
            return 0
    def fit(self):
        r = self.Y.copy()
        for i in range(self.max_iter):
            beta = self.beta.copy() # 用于计算更新大小
            for j in range(self.n_feature):
                z = np.dot(self.Xs[:,j],r)/self.n + self.beta[j]
                delta = self.soft_threshold(z) - self.beta[j]
                self.beta[j] += delta
                r -= np.dot(self.Xs[:,j], delta)
            if self.L2_norm(self.beta - beta) < self.tol:
                self.beta = self.unstandardized()
                break
    def predict(self, newdata):
        return np.dot(newdata, self.beta[1:]) + self.beta[0]

In [44]:
lasso = Lasso(Y = y_train,
              X = X_train,
              lambda_ = 0.5,
              max_iter = 10000)
lasso.fit()
half_mean_squared_error(lasso.predict(X_test),y_test)

1755.9332864742607

把第二个系数惩罚到了0的效果还是不错的，

### 岭回归-L2正则

在最小二乘回归的损失函数的基础上添加$L_2$正则项，即$\min_{\boldsymbol{w}} \frac{1}{2n}|| \boldsymbol{X} \boldsymbol{w} - \boldsymbol{y}||_2^2 + \alpha ||\boldsymbol{w}||_2^2$

可以直接计算出解析解：$\boldsymbol{w}_{opt} = (\boldsymbol{X}^T\boldsymbol{X}+2n\alpha I)^{-1}\boldsymbol{X}^T\boldsymbol{Y}$.

为了让惩罚能够公平分配，可以事先对 𝑋 进行中心标准化：
![avatar](./images/center_scale.png)

In [15]:
# X 中心标准化
center = np.mean(X_train,axis = 0)
scale = np.std(X_train,axis = 0)
center,scale
Xs = (X_train-center)/scale
center_y = np.mean(y_train)
# y 中心化
ys = y_train - center_y
# 岭回归
I = np.diag(np.ones((Xs.shape[1],)))
alpha = 1/(4*n)
w_tilde = np.linalg.inv(Xs.T.dot(Xs)+2 * n * alpha * I).dot(Xs.T).dot(ys)
# 解中心化
w_ridge = np.zeros(X.shape[1])
w_ridge[0] = center_y - w_tilde.dot(center/scale)
w_ridge[1:] = w_tilde/scale # 注意[1:-1]取不到最后一个，左闭右开
w_ridge

array([ 1.52641748e+02, -1.41409472e-01, -2.94539037e+02,  4.75592857e+02,
        3.97123753e+02, -1.71924571e+02, -1.11537838e+02, -1.68691304e+02,
        2.58964637e+02,  3.66072891e+02,  1.23512050e+02])

In [16]:
half_mean_squared_error(y_test,X_test_with_intercept.dot(w_ridge))

1763.462583930102

In [56]:
# 封装为类
class Ridge:
    def __init__(self, Y, X, lambda_):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.lambda_ = lambda_
        self.center, self.scale, self.Xs, self.center_y, self.ys = self.standardized()
        self.beta = np.zeros(self.n_feature)
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        center_y = np.mean(self.Y)
        ys = self.Y - center_y
        return (center, scale, Xs, center_y, ys)
    def fit(self):
        I = np.diag(np.ones(self.n_feature))
        self.beta = (np.linalg.inv(self.Xs.T.dot(self.Xs)+2 * self.n * self.lambda_ * I)
                      .dot(self.Xs.T).dot(self.ys))
        self.beta = self.unstandardized()
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = self.center_y - np.sum(self.beta/self.scale*self.center)
        beta[1:] = self.beta/self.scale
        return beta
    def predict(self, newdata):
        return np.dot(newdata, self.beta[1:]) + self.beta[0]

In [64]:
ridge = Ridge(Y = y_train,
              X = X_train,
              lambda_ = 0.0001)
ridge.fit()
half_mean_squared_error(ridge.predict(X_test),y_test)

1762.7157154890517

In [65]:
ridge.beta

array([ 1.52637850e+02, -2.33633967e-01, -2.95329816e+02,  4.76039266e+02,
        3.97777896e+02, -1.85874195e+02, -1.01621410e+02, -1.62064138e+02,
        2.62177007e+02,  3.70942023e+02,  1.23328645e+02])

岭回归没有将系数收缩到0的能力，只能将全部系数都收缩一部分，用处并不大。感觉最大的作用是为了让数据阵非奇异便于显式求解。

### 弹性网-L1正则和L2正则混合

在最小二乘回归的损失函数的基础上同时添加 $L_1$ 和 $L_2$ 正则项，$\min_{\boldsymbol{w}} { \frac{1}{2n}} ||\boldsymbol{X} \boldsymbol{w} - \boldsymbol{y}||_2 ^ 2 + \alpha \rho ||\boldsymbol{w}||_1 +
\frac{\alpha(1-\rho)}{2} ||\boldsymbol{w}||_2 ^ 2$，可以用坐标下降法求得最优解。

![avatar](./images/elastic_net.png)

In [17]:
#### 坐标下降法解弹性网
# X 中心标准化
center = np.mean(X_train,axis = 0)
scale = np.std(X_train,axis = 0)
Xs = (X_train-center)/scale
# y 中心化
center_y = np.mean(y_train)
ys = y_train - center_y
# 定义软阈值函数
def soft_threshold(z,l):
    if z > l:
        return z - l
    elif z < - l:
        return z + l
    else:
        return 0
# 初始化参数与残差
n,p = Xs.shape
w = np.zeros(p)
w_elanet = np.zeros(X.shape[1])
r = ys.copy()
# 设置超参数
lambda_ = 0.1
rho = 0.9
# 记录损失函数变化
l = [None] * 2002
l[0] = 0
# 坐标下降
for i in range(2001):
    for j in range(p):
        z = np.dot(Xs[:,j],r)/n + w[j]
        delta = soft_threshold(z,rho * lambda_)/((1 - rho) * lambda_ + 1) - w[j]
        w[j] += delta
        r -= np.dot(Xs[:,j], delta)
    w_elanet[0] = center_y - w.dot(center/scale)
    w_elanet[1:] = w/scale
    l[i+1] = (half_mean_squared_error(X.dot(w_elanet),y_train) + rho*lambda_*np.sum(np.abs(w_elanet)) +
              (1-rho)/2*lambda_*np.sum(w_elanet**2))
    if i % 300 == 0:
        print(f"第{i+1}次循环，参数值为 w = {w_elanet},损失值为 {l[i+1]}")
    if abs(l[i+1]-l[i])<1e-10:
        print(f"第{i+1}次循环，参数值为 w = {w_elanet},损失值为 {l[i+1]}")
        break

第1次循环，参数值为 w = [ 153.1428127   299.32083265   22.4564557   818.96573439  304.92102262
  -84.44412477  -20.22032387 -258.74452497   65.27810477  166.332269
  -37.90277575],损失值为 6770.413788018659
第301次循环，参数值为 w = [ 152.65825068    0.         -286.09518234  471.93360217  391.55225975
 -119.9295747  -138.07371445 -196.69021498  233.76914824  350.2521653
  123.09819985],损失值为 5279.592146294225
第601次循环，参数值为 w = [ 152.65825052    0.         -286.09518391  471.93360045  391.55226162
 -119.92999878 -138.07338089 -196.69003474  233.76918927  350.25231843
  123.09820158],损失值为 5279.59248626117
第627次循环，参数值为 w = [ 152.65825052    0.         -286.09518391  471.93360045  391.55226162
 -119.92999878 -138.07338088 -196.69003474  233.76918927  350.25231843
  123.09820158],损失值为 5279.592486265368


In [18]:
# 测试集损失
half_mean_squared_error(X_test_with_intercept.dot(w_elanet),y_test)

1762.2203135775587

In [74]:
# 封装为类
class ElasticNet:
    def __init__(self, Y, X, lambda_, rho, max_iter = 1000, tol = 1e-4):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.lambda_ = lambda_
        self.rho = rho
        self.max_iter = max_iter
        self.tol = tol
        self.beta = np.zeros(self.n_feature)
        self.center, self.scale, self.Xs, self.center_y, self.ys = self.standardized()
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        center_y = np.mean(self.Y)
        ys = self.Y - center_y
        return (center, scale, Xs, center_y, ys)
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = self.center_y - np.sum(self.beta/self.scale*self.center)
        beta[1:] = self.beta/self.scale
        return beta
    @staticmethod
    def L2_norm(x):
        return np.sqrt(np.sum(x**2))
    @staticmethod
    def soft_threshold(z, lambda_):
        if z < lambda_:
            return z + lambda_
        elif z > lambda_:
            return z - lambda_
        else:
            return 0
    def fit(self):
        r = self.Y.copy()
        for i in range(self.max_iter):
            beta = self.beta.copy() # 用于计算更新大小
            for j in range(self.n_feature):
                z = np.dot(self.Xs[:,j],r)/self.n + self.beta[j]
                delta = (self.soft_threshold(z, self.rho * self.lambda_)
                         /((1 - self.rho) * self.lambda_ + 1) - self.beta[j])
                self.beta[j] += delta
                r -= np.dot(self.Xs[:,j], delta)
            if self.L2_norm(self.beta - beta) < self.tol:
                self.beta = self.unstandardized()
                break
    def predict(self, newdata):
        return np.dot(newdata, self.beta[1:]) + self.beta[0]

In [82]:
elanet = ElasticNet(Y = y_train,
                    X = X_train,
                    lambda_ = 0.1,
                    rho = 0.9,
                    max_iter = 10000)
elanet.fit()
half_mean_squared_error(elanet.predict(X_test),y_test)

1762.3624533635073

感觉弹性网并不会比lasso好，大系数并不需要收缩，只需要收缩小系数。而岭回归按比例收缩所有系数并不合理。

### MCP惩罚

非凹惩罚，解决了lasso等惩罚项对不需要收缩的系数带来有偏性的问题。

![avatar](./images/MCP.png)

可以看到，当系数超过某个值时，它将会与没有惩罚时的结果一样（当然，这只是一维的情况，只能说这个惩罚改善了有偏的程度）。

In [79]:
# MCP 的推荐值
2/(1-np.max(X.T.dot(X)-np.diag(np.diag(X.T.dot(X)))))

5.359322926513111

In [80]:
#### 坐标下降法解 MCP 惩罚项的损失函数
# X 中心标准化
center = np.mean(X_train,axis = 0)
scale = np.std(X_train,axis = 0)
Xs = (X_train-center)/scale
# y 中心化
center_y = np.mean(y_train)
ys = y_train - center_y
# 定义软阈值函数
def soft_threshold(z,l):
    if z > l:
        return z - l
    elif z < - l:
        return z + l
    else:
        return 0
# 定义 MCP 的惩罚函数
def MCP(w,lam,gamma):
    penalty = np.zeros_like(w)
    for i in range(len(w)):
        if abs(w[i]) <= gamma * lam:
            penalty[i] = lam * w[i] - w[i]**2/(2*gamma)
        else:
            penalty[i] = gamma * lam ** 2/2
    return np.sum(penalty)
# 定义 MCP 的阈值函数
def firm_threshold(z,l,gamma):
    if abs(z) <= gamma * l:
        return soft_threshold(z,l)/(1-1/(gamma))
    else:
        return z
# 初始化参数与残差
n,p = Xs.shape
w = np.zeros(p)
w_mcp = np.zeros(X.shape[1])
r = ys.copy()
# 设置超参数
lambda_ = 2
gamma = 5.36 # MCP惩罚推荐值的近似值
# 记录损失函数变化
l = [None] * 1002
l[0] = 0
# 坐标下降
for i in range(1001):
    for j in range(p):
        z = np.dot(Xs[:,j],r)/n + w[j]
        delta = firm_threshold(z, lambda_, gamma) - w[j]
        w[j] += delta
        r -= np.dot(Xs[:,j], delta)
    w_mcp[0] = center_y - w.dot(center/scale)
    w_mcp[1:] = w/scale
    l[i+1] = half_mean_squared_error(X.dot(w_mcp),y_train) + MCP(w_mcp,lambda_,gamma)
    if i % 100 == 0:
        print(f"第{i+1}次循环，参数值为 w = {w_mcp},损失值为 {l[i+1]}")
    if abs(l[i+1]-l[i])<1e-10:
        print(f"第{i+1}次循环，参数值为 w = {w_mcp},损失值为 {l[i+1]}")
        break

第1次循环，参数值为 w = [ 153.26202658  304.17464389    0.          830.35875114  309.08565146
  -61.97987419   -3.84910694 -265.98744374    0.          175.54115202
    0.        ],损失值为 1707.507800391112
第38次循环，参数值为 w = [ 152.64035268    0.         -283.5273765   471.9887538   395.63734044
    0.         -108.29805955 -365.55559342    0.          372.55173407
   90.31263603],损失值为 1424.6629310473581


In [81]:
half_mean_squared_error(X_test_with_intercept.dot(w_mcp),y_test)

1733.6348912070428

In [84]:
# 封装为类
class MCP:
    def __init__(self, Y, X, lambda_, gamma, max_iter = 1000, tol = 1e-4):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.lambda_ = lambda_
        self.gamma = gamma
        self.max_iter = max_iter
        self.tol = tol
        self.beta = np.zeros(self.n_feature)
        self.center, self.scale, self.Xs, self.center_y, self.ys = self.standardized()
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        center_y = np.mean(self.Y)
        ys = self.Y - center_y
        return (center, scale, Xs, center_y, ys)
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = self.center_y - np.sum(self.beta/self.scale*self.center)
        beta[1:] = self.beta/self.scale
        return beta
    @staticmethod
    def L2_norm(x):
        return np.sqrt(np.sum(x**2))
    @staticmethod
    def soft_threshold(z, lambda_):
        if z < -lambda_:
            return z + lambda_
        elif z > lambda_:
            return z - lambda_
        else:
            return 0
    def firm_threshold(self, z):
        if abs(z) <= self.gamma * self.lambda_:
            return self.soft_threshold(z, self.lambda_)/(1-1/(self.gamma))
        else:
            return z
    def fit(self):
        r = self.Y.copy()
        for i in range(self.max_iter):
            beta = self.beta.copy() # 用于计算更新大小
            for j in range(self.n_feature):
                z = np.dot(self.Xs[:,j],r)/self.n + self.beta[j]
                delta = self.firm_threshold(z) - self.beta[j]
                self.beta[j] += delta
                r -= np.dot(self.Xs[:,j], delta)
            if self.L2_norm(self.beta - beta) < self.tol:
                self.beta = self.unstandardized()
                break
    def predict(self, newdata):
        return np.dot(newdata, self.beta[1:]) + self.beta[0]

In [85]:
mcp = MCP(Y = y_train,
          X = X_train,
          lambda_ = 2,
          gamma = 5.36,
          max_iter = 10000)
mcp.fit()
half_mean_squared_error(mcp.predict(X_test),y_test)

1733.6348267933677

In [86]:
mcp.beta

array([ 152.64035123,    0.        , -283.52711268,  471.9886988 ,
        395.6366227 ,    0.        , -108.29877561, -365.55505158,
          0.        ,  372.55186116,   90.31334276])

可以看到，MCP惩罚收缩了三个变量，但预测效果反而更好。

### SCAD 惩罚

非凹惩罚，解决了lasso等惩罚项对不需要收缩的系数带来有偏性的问题，和MCP惩罚类似。

![avatar](./images/SCAD.png)

In [22]:
#### 坐标下降法解 SCAD 惩罚项的损失函数
# X 中心标准化
center = np.mean(X_train,axis = 0)
scale = np.std(X_train,axis = 0)
Xs = (X_train-center)/scale
# y 中心化
center_y = np.mean(y_train)
ys = y_train - center_y
# 定义软阈值函数
def soft_threshold(z,l):
    if z > l:
        return z - l
    elif z < - l:
        return z + l
    else:
        return 0
# 定义 SCAD 的惩罚函数
def SCAD(w,lam,gamma):
    penalty = np.zeros_like(w)
    for i in range(len(w)):
        if abs(w[i]) <= lam:
            penalty[i] = lam * w[i]
        elif abs(w[i]) <= gamma * lam:
            penalty[i] = (gamma * lam * w[i] - (w[i] ** 2 + lam ** 2)/2)/(gamma - 1)
        else:
            penalty[i] = lam ** 2 * (gamma + 1) / 2
    return np.sum(penalty)
# 定义 SCAD 的阈值函数
def scad_threshold(z,l,gamma):
    if abs(z) <= 2 * l:
        return soft_threshold(z,l)
    elif abs(z) <= gamma * l:
        return soft_threshold(z,gamma*l/(gamma-1))/(1-1/(gamma-1))
    else:
        return z
# 初始化参数与残差
n,p = Xs.shape
w = np.zeros(p)
w_scad = np.zeros(X.shape[1])
r = ys.copy()
# 设置超参数
lambda_ = 2
gamma = 3.7 # SCAD惩罚推荐值
# 记录损失函数变化
l = [None] * 1002
l[0] = 0
# 坐标下降
for i in range(1001):
    for j in range(p):
        z = np.dot(Xs[:,j],r)/n + w[j]
        delta = scad_threshold(z, lambda_, gamma) - w[j]
        w[j] += delta
        r -= np.dot(Xs[:,j], delta)
    w_scad[0] = center_y - w.dot(center/scale)
    w_scad[1:] = w/scale
    l[i+1] = half_mean_squared_error(X.dot(w_scad),y_train) + SCAD(w_scad,lambda_,gamma)
    if i % 100 == 0:
        print(f"第{i+1}次循环，参数值为 w = {w_scad},损失值为 {l[i+1]}")
    if abs(l[i+1]-l[i])<1e-10:
        print(f"第{i+1}次循环，参数值为 w = {w_scad},损失值为 {l[i+1]}")
        break

第1次循环，参数值为 w = [ 153.27671749  304.17464389    0.          830.35875114  309.08565146
  -55.22351125   -9.22088028 -267.47816827    0.          183.80335675
    0.        ],损失值为 1716.5152924010367
第43次循环，参数值为 w = [ 152.61947083    0.         -283.67119282  472.45780798  394.01802161
    0.         -118.92422474 -365.42325474    0.          373.39026203
   97.67026306],损失值为 1413.287260907767


In [23]:
half_mean_squared_error(X_test_with_intercept.dot(w_scad),y_test)

1735.9917114873363

In [88]:
# 封装为类
class SCAD:
    def __init__(self, Y, X, lambda_, gamma = 3.7, max_iter = 1000, tol = 1e-4):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.lambda_ = lambda_
        self.gamma = gamma
        self.max_iter = max_iter
        self.tol = tol
        self.beta = np.zeros(self.n_feature)
        self.center, self.scale, self.Xs, self.center_y, self.ys = self.standardized()
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        center_y = np.mean(self.Y)
        ys = self.Y - center_y
        return (center, scale, Xs, center_y, ys)
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = self.center_y - np.sum(self.beta/self.scale*self.center)
        beta[1:] = self.beta/self.scale
        return beta
    @staticmethod
    def L2_norm(x):
        return np.sqrt(np.sum(x**2))
    @staticmethod
    def soft_threshold(z, lambda_):
        if z < -lambda_:
            return z + lambda_
        elif z > lambda_:
            return z - lambda_
        else:
            return 0
    def scad_threshold(self, z):
        if abs(z) <= 2 * self.lambda_:
            return self.soft_threshold(z, self.lambda_)
        elif abs(z) <= self.gamma * self.lambda_:
            return self.soft_threshold(z,self.gamma*self.lambda_/(self.gamma-1))/(1-1/(self.gamma-1))
        else:
            return z
    def fit(self):
        r = self.Y.copy()
        for i in range(self.max_iter):
            beta = self.beta.copy() # 用于计算更新大小
            for j in range(self.n_feature):
                z = np.dot(self.Xs[:,j],r)/self.n + self.beta[j]
                delta = self.scad_threshold(z) - self.beta[j]
                self.beta[j] += delta
                r -= np.dot(self.Xs[:,j], delta)
            if self.L2_norm(self.beta - beta) < self.tol:
                self.beta = self.unstandardized()
                break
    def predict(self, newdata):
        return np.dot(newdata, self.beta[1:]) + self.beta[0]

In [90]:
scad = SCAD(Y = y_train,
            X = X_train,
            lambda_ = 2,
            max_iter = 10000)
scad.fit()
half_mean_squared_error(scad.predict(X_test),y_test)

1735.9917947559345

和MCP惩罚的效果类似。

## 树模型

### CART树

![avatar](./images/decision_tree1.png)
![avatar](./images/decision_tree2.png)

In [114]:
# 回归树
class Tree:
    def __init__(self, Y, X, min_node_size, max_node_depth, rule = None):
        # 数据存储
        self.Y = Y
        self.X = X
        self.n, self.feature_num = X.shape
        # 递归控制
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.n = len(Y)
        # 节点分裂信息
        self.mse = np.var(Y)
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        # 节点预测值
        self.yhat = np.mean(Y)
        # 字符串记录，可以用来调试
        self.rule = rule if rule is not None else ""
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        for feature in range(self.feature_num):
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:, feature] < value
                ridx = self.X[:, feature] >= value
                n_left = np.sum(lidx)
                n_right = np.sum(ridx)
                # 注意这里只是跳出本次循环
                if (n_left == 0) or (n_right == 0):
                    continue
                mse_left = np.var(self.Y[lidx])
                mse_right = np.var(self.Y[ridx])
                mse_gain = self.mse - (n_left * mse_left + n_right * mse_right) / self.n
                if mse_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = mse_gain
        return (best_feature, best_value)
    def grow_tree(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:, self.best_feature] < self.best_value
                ridx = self.X[:, self.best_feature] >= self.best_value
                left = Tree(Y = self.Y[lidx],
                            X = self.X[lidx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                left.grow_tree()
                right = Tree(Y = self.Y[ridx],
                             X = self.X[ridx],
                             min_node_size = self.min_node_size,
                             max_node_depth = self.max_node_depth - 1,
                             rule = f"X{self.best_feature+1} >= {self.best_value}")
                right.grow_tree()
                self.left = left
                self.right = right
    def predict(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.yhat
        return yhat

In [141]:
cart = Tree(Y = y_train,
            X = X_train,
            min_node_size = 10,
            max_node_depth = 5)
cart.grow_tree()

In [142]:
half_mean_squared_error(cart.predict(X_test),y_test)

1993.1437245941881

单棵决策树效果并不怎么样，不过可以通过集成模型来提高预测效果，集成模型有两族，bagging和boosting。

### bagging

#### 随机森林

In [179]:
# 随机森林的基学习器
class Tree:
    def __init__(self, Y, X, min_node_size, max_node_depth, mtry, rule = None):
        # 数据存储
        self.Y = Y
        self.X = X
        self.n, self.feature_num = X.shape
        # 递归控制
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.n = len(Y)
        # 节点分裂信息
        self.mtry = mtry
        self.mse = np.var(Y)
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        # 节点预测值
        self.yhat = np.mean(Y)
        # 字符串记录，可以用来调试
        self.rule = rule if rule is not None else ""
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        feature_try = np.random.choice(np.arange(self.feature_num), self.mtry, replace = False)
        for feature in feature_try:
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:, feature] < value
                ridx = self.X[:, feature] >= value
                n_left = np.sum(lidx)
                n_right = np.sum(ridx)
                # 注意这里只是跳出本次循环
                if (n_left == 0) or (n_right == 0):
                    continue
                mse_left = np.var(self.Y[lidx])
                mse_right = np.var(self.Y[ridx])
                mse_gain = self.mse - (n_left * mse_left + n_right * mse_right) / self.n
                if mse_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = mse_gain
        return (best_feature, best_value)
    def grow_tree(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:, self.best_feature] < self.best_value
                ridx = self.X[:, self.best_feature] >= self.best_value
                left = Tree(Y = self.Y[lidx],
                            X = self.X[lidx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            mtry = self.mtry,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                left.grow_tree()
                right = Tree(Y = self.Y[ridx],
                             X = self.X[ridx],
                             min_node_size = self.min_node_size,
                             max_node_depth = self.max_node_depth - 1,
                             mtry = self.mtry,
                             rule = f"X{self.best_feature+1} >= {self.best_value}")
                right.grow_tree()
                self.left = left
                self.right = right
    def predict(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.yhat
        return yhat

In [180]:
from tqdm import tqdm
# 随机森林
class RandomForest:
    def __init__(self, Y, X, min_node_size, max_node_depth, mtry, ntree):
        self.Y = Y
        self.X = X
        self.n, self.feature_num = X.shape
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.mtry = mtry
        self.ntree = ntree
        self.rf = []
    def fit(self):
        for i in tqdm(range(self.ntree)):
            bootstrap_index = np.random.choice(self.n, self.n, replace = True)
            X, Y = self.X[bootstrap_index],self.Y[bootstrap_index]
            tree = Tree(Y = Y,
                        X = X,
                        min_node_size = self.min_node_size,
                        max_node_depth = self.max_node_depth,
                        mtry = self.mtry)
            tree.grow_tree()
            self.rf.append(tree)
    def predict(self, newdata):
        n = newdata.shape[0]
        tmp = np.zeros((self.ntree,n))
        for i in range(self.ntree):
            tmp[i,:] = self.rf[i].predict(newdata)
        return np.mean(tmp,axis = 0)

In [181]:
np.random.seed(521)
rf = RandomForest(Y = y_train,
                  X = X_train,
                  min_node_size = 10,
                  max_node_depth = 4,
                  mtry = 4,
                  ntree = 100)
rf.fit()
half_mean_squared_error(rf.predict(X_test), y_test)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.16it/s]


1732.7680518928373

效果比较显著。

### boosting

#### GBDT

In [183]:
# GBDT的基学习器，和最开始那棵决策树一样，只不过集成时传的Y是残差
class Tree:
    def __init__(self, Y, X, min_node_size, max_node_depth, rule = None):
        # 数据存储
        self.Y = Y
        self.X = X
        self.n, self.feature_num = X.shape
        # 递归控制
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.n = len(Y)
        # 节点分裂信息
        self.mse = np.var(Y)
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        # 节点预测值
        self.yhat = np.mean(Y)
        # 字符串记录，可以用来调试
        self.rule = rule if rule is not None else ""
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        for feature in range(self.feature_num):
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:, feature] < value
                ridx = self.X[:, feature] >= value
                n_left = np.sum(lidx)
                n_right = np.sum(ridx)
                # 注意这里只是跳出本次循环
                if (n_left == 0) or (n_right == 0):
                    continue
                mse_left = np.var(self.Y[lidx])
                mse_right = np.var(self.Y[ridx])
                mse_gain = self.mse - (n_left * mse_left + n_right * mse_right) / self.n
                if mse_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = mse_gain
        return (best_feature, best_value)
    def grow_tree(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:, self.best_feature] < self.best_value
                ridx = self.X[:, self.best_feature] >= self.best_value
                left = Tree(Y = self.Y[lidx],
                            X = self.X[lidx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                left.grow_tree()
                right = Tree(Y = self.Y[ridx],
                             X = self.X[ridx],
                             min_node_size = self.min_node_size,
                             max_node_depth = self.max_node_depth - 1,
                             rule = f"X{self.best_feature+1} >= {self.best_value}")
                right.grow_tree()
                self.left = left
                self.right = right
    def predict(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.yhat
        return yhat

In [184]:
class GBDT:
    def __init__(self, Y, X, min_node_size, max_node_depth, nboost, lr = 1):
        self.Y = Y
        self.X = X
        self.n = X.shape[0]
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.nboost = nboost
        self.lr = lr
        self.ybar = np.mean(Y)
        self.base_pred = np.mean(Y)
        self.tree_set = []
    def fit(self):
        for i in tqdm(range(self.nboost)):
            res = self.Y - self.base_pred
            tree = Tree(Y = res,
                        X = self.X,
                        min_node_size = self.min_node_size,
                        max_node_depth = self.max_node_depth)
            tree.grow_tree()
            self.base_pred += self.lr * tree.predict(self.X)
            self.tree_set.append(tree)
    def predict(self, newdata):
        n = newdata.shape[0]
        yhat = self.ybar
        for tree in self.tree_set:
            yhat += self.lr * tree.predict(newdata)
        return yhat

In [190]:
gbdt = GBDT(Y = y_train,
            X = X_train,
            min_node_size = 10,
            max_node_depth = 3,
            nboost = 100,
            lr = 0.05)
gbdt.fit()
half_mean_squared_error(gbdt.predict(X_test),y_test)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:25<00:00,  3.95it/s]


1796.1243293687164

感觉提升不上去啊。。。

#### XGboost

In [191]:
# XGboost的基学习器
class Tree:
    def __init__(self, X, grad, hess, min_node_size, max_node_depth, mtry, lambda_ = 1, gamma = 1, 
                 eps = 0.1, rule = None):
        self.X = X
        self.n, self.feature_num = X.shape
        self.grad = grad
        self.hess = hess
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.mtry = mtry
        self.lambda_ = lambda_
        self.gamma = gamma
        self.eps = eps
        self.score = self.compute_score(self.grad, self.hess) #叶节点得分
        self.rule = rule if rule is not None else ""
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        self.leaf_weight = self.compute_weight(self.grad, self.hess)
    def compute_score(self, grad, hess):
        return -1/2 * np.sum(grad)**2 / (np.sum(hess) + self.lambda_)
    def compute_weight(self, grad, hess):
        return - np.sum(grad) / (np.sum(hess) + self.lambda_)
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        # Exact Greedy Algorithm
        feature_try = np.random.choice(np.arange(self.feature_num), self.mtry, replace = False)
        for feature in feature_try:
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:, feature] < value
                ridx = self.X[:, feature] >= value
                lscore = self.compute_score(self.grad[lidx], self.hess[lidx])
                rscore = self.compute_score(self.grad[ridx], self.hess[ridx])
                score_gain = (self.score - lscore - rscore)/2 - self.gamma
                if score_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = score_gain
        return (best_feature, best_value)
    def grow_tree(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:, self.best_feature] < self.best_value
                ridx = self.X[:, self.best_feature] >= self.best_value
                left = Tree(X = self.X[lidx],
                            grad = self.grad[lidx],
                            hess = self.hess[lidx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            mtry = self.mtry,
                            lambda_ = self.lambda_,
                            gamma = self.gamma,
                            eps = self.eps,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                left.grow_tree()
                right = Tree(X = self.X[ridx],
                            grad = self.grad[ridx],
                            hess = self.hess[ridx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            mtry = self.mtry,
                            lambda_ = self.lambda_,
                            gamma = self.gamma,
                            eps = self.eps,
                            rule = f"X{self.best_feature+1} >= {self.best_value}")
                right.grow_tree()
                self.left = left
                self.right = right
    def predict(self, newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.leaf_weight
        return yhat

In [192]:
from tqdm import tqdm
class XgboostRegressor:
    def __init__(self, X, Y, min_node_size, max_node_depth, mtry, nboost, lambda_ = 1, gamma = 1, eps = 0.1, lr = 1):
        self.X = X
        self.n,self.p = X.shape
        self.Y = Y
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.mtry = mtry
        self.nboost = nboost
        self.lambda_ = lambda_
        self.gamma = gamma
        self.eps = eps
        self.lr = lr
        self.tree_set = []
        self.base_pred = np.mean(Y)
    
    def fit(self):
        for i in tqdm(range(self.nboost)):
            grad = self.base_pred - self.Y
            hess = np.ones_like(grad)
            tree = Tree(X = self.X,
                        grad = grad,
                        hess = hess,
                        min_node_size = self.min_node_size,
                        max_node_depth = self.max_node_depth,
                        mtry = self.mtry,
                        lambda_ = self.lambda_,
                        gamma = self.gamma,
                        eps = self.eps)
            tree.grow_tree()
            self.base_pred += self.lr * tree.predict(self.X)
            self.tree_set.append(tree)
    def predict(self,newdata):
        n = newdata.shape[0]
        yhat = np.ones(n) * np.mean(self.Y)
        for tree in self.tree_set:
            yhat += self.lr * tree.predict(newdata)
        return yhat

In [196]:
np.random.seed(521)
xgbooster = XgboostRegressor(X = X_train,
                             Y = y_train,
                             min_node_size = 10,
                             max_node_depth = 4,
                             mtry = 4,
                             nboost = 100,
                             gamma = 1,
                             lr = 0.05)
xgbooster.fit()
half_mean_squared_error(xgbooster.predict(X_test),y_test)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.90it/s]


1697.0190564794805

提上去了！

# 分类

## 逻辑回归

### 不带正则项的逻辑回归

In [5]:
# Adam优化算法
class Adam:
    def __init__(self, lr = 0.002, beta1 = 0.9, beta2 = 0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.m = None
        self.v = None
        self.um = None
        self.uv = None
        self.iter = 0
    def update(self, params, grads):
        if self.m is None:
            self.m = np.zeros_like(grads)
            self.v = np.zeros_like(grads)
            self.um = np.zeros_like(grads)
            self.uv = np.zeros_like(grads)
        self.iter += 1
        self.m = self.beta1 * self.m + (1.0 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1.0 - self.beta2) * grads ** 2
        self.um = self.m / (1 - self.beta1 ** self.iter)
        self.uv = self.v / (1 - self.beta2 ** self.iter)
        params -= self.lr * self.um / (np.sqrt(self.uv) + 1e-7)

In [6]:
class LR:
    def __init__(self, Y, X, optimizer = Adam, max_iter = 1000, tol = 1e-5):
        self.Y = Y
        self.X = X
        self.n,self.n_feature = X.shape
        self.optimizer = optimizer
        self.beta = np.zeros(self.n_feature)
        self.P = self.sigmoid(self.X,self.beta)
        self.tol = tol
        self.max_iter = max_iter
    @staticmethod
    def sigmoid(x,beta):
        return 1/(1+np.exp(-np.dot(x,beta)))
    def fit(self):
        for i in range(self.max_iter):
            grads = np.dot(self.X.T, self.P-self.Y)
            beta_old = self.beta.copy()
            self.optimizer.update(self.beta, grads)
            self.P = self.sigmoid(self.X,self.beta)
            if np.sum((self.beta - beta_old)**2) < self.tol:
                break
    def predict_prob(self,newdata):
        return self.sigmoid(newdata, self.beta)
    def predict(self,newdata,threshold = 1e-4):
        P = self.predict_prob(newdata)
        return np.where(P>=threshold+0.5,1,0)

In [3]:
import numpy as np
import pandas as pd
from sklearn import datasets
X,y = datasets.load_breast_cancer(return_X_y=True)
# 划分训练集和测试集
np.random.seed(104)
index1 = np.random.choice(X.shape[0],round(X.shape[0]*0.7),replace = False)
index2 = np.array(list(set(range(X.shape[0]))-set(index1)))
X_train, X_test, y_train, y_test = X[index1,].copy(), X[index2,].copy(), y[index1].copy(), y[index2].copy()

In [7]:
adam = Adam(lr=0.1)
logistic_reg = LR(Y = y_train,
                  X = X_train,
                  optimizer=adam)
logistic_reg.fit()
np.mean(logistic_reg.predict(X_test) == y_test)

0.9590643274853801

### 带L1惩罚的逻辑回归

In [8]:
class LR_lasso:
    def __init__(self, Y, X, max_iter, lambda_, tol = 1e-4):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.max_iter = max_iter
        self.lambda_ = lambda_
        self.tol = tol
        self.center,self.scale,self.Xs = self.standardized()
        self.beta = np.zeros(self.n_feature)
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        return (center, scale, Xs)
    @staticmethod
    def L2_norm(x,y):
        return np.sqrt(np.sum((x-y)**2))
    @staticmethod
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    @staticmethod
    def soft_threshold(z, lambda_):
        if z < -lambda_:
            re = z + lambda_
        elif z > lambda_:
            re = z - lambda_
        else:
            re = 0
        return re
    
    def fit(self):
        for i in range(self.max_iter):
            beta = self.beta.copy()
            for j in range(self.n_feature):
                P = self.sigmoid(np.dot(self.Xs, beta))
                grad = np.dot(self.Xs.T, P - self.Y)
                V = np.sum(self.Xs[:,j] * P * (1-P) * self.Xs[:,j])
                z = beta[j] - grad[j]/V
                beta[j] = self.soft_threshold(z, self.lambda_/V)
            if (self.L2_norm(beta,self.beta) > self.tol):
                self.beta = beta.copy()
            else:
                self.beta = self.unstandardized()
                break
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = -np.sum(self.center * self.beta / self.scale)
        beta[1:] = self.beta/self.scale
        return beta
    def predict_prob(self,newdata):
        tmp = np.dot(newdata, self.beta[1:])+self.beta[0]
        return self.sigmoid(tmp)
    def predict(self, newdata, threshold = 1e-4):
        P = self.predict_prob(newdata)
        return np.where(P > (0.5+threshold), 1, 0)

In [10]:
logistic_reg_lasso = LR_lasso(Y = y_train,
                              X = X_train,
                              max_iter=1000,
                              lambda_=5)
logistic_reg_lasso.fit()
np.mean(logistic_reg_lasso.predict(X_test) == y_test)

0.9824561403508771

### 带MCP惩罚的逻辑回归

In [11]:
class LR_MCP:
    def __init__(self, Y, X, max_iter, lambda_, gamma, tol = 1e-4):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.max_iter = max_iter
        self.lambda_ = lambda_
        self.tol = tol
        self.gamma = gamma
        self.center,self.scale,self.Xs = self.standardized()
        self.beta = np.zeros(self.n_feature)
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        return (center, scale, Xs)
    @staticmethod
    def L2_norm(x,y):
        return np.sqrt(np.sum((x-y)**2))
    @staticmethod
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    @staticmethod
    def soft_threshold(z, lambda_):
        if z < -lambda_:
            re = z + lambda_
        elif z > lambda_:
            re = z - lambda_
        else:
            re = 0
        return re
    def firm_threshold(self, z, v):
        if (abs(z) < self.gamma * self.lambda_):
            return self.soft_threshold(z, self.lambda_/v)/(1-1/(self.gamma * v))
        else:
            return z
    def fit(self):
        for i in range(self.max_iter):
            beta = self.beta.copy()
            for j in range(self.n_feature):
                P = self.sigmoid(np.dot(self.Xs, beta))
                grad = np.dot(self.Xs.T, P - self.Y)
                V = np.sum(self.Xs[:,j] * P * (1-P) * self.Xs[:,j])
                z = beta[j] - grad[j]/V
                beta[j] = self.firm_threshold(z, V)
            if (self.L2_norm(beta,self.beta) > self.tol):
                self.beta = beta.copy()
            else:
                self.beta = self.unstandardized()
                break
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = -np.sum(self.center * self.beta / self.scale)
        beta[1:] = self.beta/self.scale
        return beta
    def predict_prob(self,newdata):
        tmp = np.dot(newdata, self.beta[1:])+self.beta[0]
        return self.sigmoid(tmp)
    def predict(self, newdata, threshold = 1e-4):
        P = self.predict_prob(newdata)
        return np.where(P > (0.5+threshold), 1, 0)

In [12]:
logistic_reg_mcp = LR_MCP(Y = y_train,
                          X = X_train,
                          max_iter = 10000,
                          lambda_ = 4,
                          gamma = 3)
logistic_reg_mcp.fit()
np.mean(logistic_reg_mcp.predict(X_test) == y_test)

0.9824561403508771

### 带SCAD惩罚的逻辑回归

In [13]:
class LR_SCAD:
    def __init__(self, Y, X, max_iter, lambda_, gamma, tol = 1e-4):
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        self.max_iter = max_iter
        self.lambda_ = lambda_
        self.tol = tol
        self.gamma = gamma
        self.center,self.scale,self.Xs = self.standardized()
        self.beta = np.zeros(self.n_feature)
    def standardized(self):
        center = np.mean(self.X,axis = 0)
        scale = np.std(self.X,axis = 0)
        Xs = (self.X-center)/scale
        return (center, scale, Xs)
    @staticmethod
    def L2_norm(x,y):
        return np.sqrt(np.sum((x-y)**2))
    @staticmethod
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    @staticmethod
    def soft_threshold(z, lambda_):
        if z < -lambda_:
            re = z + lambda_
        elif z > lambda_:
            re = z - lambda_
        else:
            re = 0
        return re
    def scad_threshold(self, z, v):
        if (abs(z) < (1+1/v) * self.lambda_):
            return self.soft_threshold(z, self.lambda_/v)
        elif (abs(z) < self.gamma * self.lambda_):
            tmp = self.soft_threshold(z, self.gamma * self.lambda_/((self.gamma-1)*v))
            return tmp/(1-1/((self.gamma-1)*v))
        else:
            return z
    def fit(self):
        for i in range(self.max_iter):
            beta = self.beta.copy()
            for j in range(self.n_feature):
                P = self.sigmoid(np.dot(self.Xs, beta))
                grad = np.dot(self.Xs.T, P - self.Y)
                V = np.sum(self.Xs[:,j] * P * (1-P) * self.Xs[:,j])
                z = beta[j] - grad[j]/V
                beta[j] = self.scad_threshold(z, V)
            if (self.L2_norm(beta,self.beta) > self.tol):
                self.beta = beta.copy()
            else:
                self.beta = self.unstandardized()
                break
    def unstandardized(self):
        beta = np.zeros(self.n_feature+1)
        beta[0] = -np.sum(self.center * self.beta / self.scale)
        beta[1:] = self.beta/self.scale
        return beta
    def predict_prob(self,newdata):
        tmp = np.dot(newdata, self.beta[1:])+self.beta[0]
        return self.sigmoid(tmp)
    def predict(self, newdata, threshold = 1e-4):
        P = self.predict_prob(newdata)
        return np.where(P > (0.5+threshold), 1, 0)

In [14]:
logistic_reg_scad = LR_SCAD(Y = y_train,
                            X = X_train,
                            max_iter = 10000,
                            lambda_ = 5,
                            gamma = 5)
logistic_reg_scad.fit()
np.mean(logistic_reg_scad.predict(X_test) == y_test)

0.9824561403508771

## 支持向量机

这个SMO算法写得实在是不怎么样，所以效果可能也不怎么样。。。

In [6]:
class SVM:
    def __init__(self, X, Y, C, epoch_num, kernel, eps = 1e-2, tol = 1e-6, degree = 3, sigma = 1):
        self.X = X
        self.Y = Y
        self.n, self.p = X.shape
        self.C = C # 惩罚系数
        self.epoch_num = epoch_num # 最大下降次数
        self.eps = eps # 最大违反KKT条件的程度
        self.tol = tol # 目标函数最小下降程度
        self.alpha = np.zeros(self.n) 
        self.b = 0
        self.kernel = kernel
        self.degree = degree
        self.sigma = sigma
        self.K = self.compute_K(self.X, self.X,) # 核变换后的内积结果
        self.g = self.compute_g() # 对每个xi的标签的预测
        self.E = self.g - self.Y  # 预测误差
    @staticmethod
    def linear_kernel(x,y):
        return np.dot(x,y) # 换核函数只需要修改它即可
    def poly_kernel(self,x,y):
        return (np.dot(x,y)+1) ** self.degree
    def gaussian_kernel(self,x,y):
        return np.exp(-np.sum((x-y)**2)/(2 * self.sigma **2))
    # 计算核函数内积矩阵
    def compute_K(self, X1, X2):
        n1 = X1.shape[0]
        n2 = X2.shape[0]
        K = np.zeros((n1, n2))
        for i in range(n1):
            for j in range(n2):
                if self.kernel == "linear":
                    K[i,j] = self.linear_kernel(X1[i],X2[j])
                elif self.kernel == "polynomial":
                    K[i,j] = self.poly_kernel(X1[i],X2[j])
                elif self.kernel == "gaussian":
                    K[i,j] = self.gaussian_kernel(X1[i],X2[j])
        return K
    # 计算g(x)
    def compute_g(self):
        g = np.zeros(self.n)
        for i in range(self.n):
            g[i] = np.dot(self.alpha * self.Y, self.K[i]) + self.b
        return g
    # 计算目标函数值
    def compute_obj(self,alpha):
        return np.dot(alpha*self.Y, self.K).dot(alpha* self.Y)/2 - np.sum(alpha)
    # 外循环中按顺序寻找i1，最先搜索支持向量，再搜索远离分隔平面的点，最后搜索软间隔中的点
    def choose_i1(self, flag1):
        value = self.Y * self.E
        is_support = (self.alpha > 0) & (self.alpha < self.C) # 支持向量
        is_beyond_support = self.alpha == 0 # 远离分隔平面
        is_between_support = self.alpha == self.C # 软间隔之间
        not_zero = ((value < -self.eps) | (value > self.eps)) # 对支持向量而言，value不为0违反KKT条件
        is_positive = value > self.eps # 对软间隔之间的点而言，value大于0违反KKT条件
        is_negative = value < -self.eps # 对远离分隔平面的点而言，value小于0违反KKT条件
        support_vio = is_support & not_zero
        beyond_support_vio = is_beyond_support & is_negative
        between_support_vio = is_between_support & is_positive
        flag2 = flag1 - np.sum(support_vio)
        flag3 = flag1 - np.sum(support_vio) - np.sum(beyond_support_vio)
        if np.sum(support_vio) >= flag1:
            tmp = np.sort(np.abs(value[support_vio]))
            i1 = np.where((np.abs(value) == tmp[-flag1]) & is_support)[0]
        elif np.sum(beyond_support_vio) >= flag2:
            tmp = np.sort(value[beyond_support_vio])[::-1]
            i1 = np.where((value == tmp[-flag2]) & is_beyond_support)[0]
        elif np.sum(between_support_vio) > flag3:
            print(前面两种都没有)
            tmp = np.sort(value[between_support_vio])
            i1 = np.where((value == tmp[-flag3]) & is_between_support)[0]
        else:
            i1 = None
        return i1
    # 内循环启发式寻找i2，这里将i2先排好序，用于一个一个找
    def choose_i2(self,i1):
        i2 = np.zeros(self.n,dtype = int)
        idx = np.arange(self.n)
        if self.E[i1] > 0:
            i2[0] = np.argmin(self.E)
        else:
            i2[0] = np.argmax(self.E)
        tmp1 = idx[(self.alpha > 0) & (self.alpha < self.C) & (idx != i1)]
        tmp2 = idx[((self.alpha == 0) | (self.alpha == self.C)) & (idx != i1)]
        i2[1:] = np.concatenate([tmp1,tmp2])
        return i2
        
    def smo(self):
        for epoch in range(self.epoch_num):
#             print(f"--------------------第{epoch}次迭代-----------------")
            out_loop = True
            flag = 1 # 用于指明i1已经搜索到哪里了
            obj = self.compute_obj(self.alpha)
            while out_loop:
                i1 = self.choose_i1(flag)
                if type(i1) is int:
                    i1 = [i1]
                # i1已经找遍，且没有再能够使函数可观下降的一对alpha，所以结束算法
                elif i1 is None:
                    return self.alpha
                # 外循环搜索i1
                for j in i1:
                    i2 = self.choose_i2(j)
                    # 内循环搜索i2
                    for k in i2:
                        eta = self.K[j,j] + self.K[k,k] - 2 * self.K[j,k]
                        alpha2_unc = self.alpha[k] + self.Y[k] * (self.E[j] - self.E[k]) / eta
                        if self.Y[j] == self.Y[k]:
                            L = max(0, self.alpha[j] + self.alpha[k] - self.C)
                            H = min(self.C, self.alpha[j] + self.alpha[k])
                        else:
                            L = max(0, self.alpha[k] - self.alpha[j])
                            H = min(self.C, self.C + self.alpha[k] - self.alpha[j])
                        if alpha2_unc > H:
                            alpha2 = H
                        elif alpha2_unc >= L:
                            alpha2 = alpha2_unc
                        else:
                            alpha2 = L
                        alpha1 = self.alpha[j] + self.Y[j]*self.Y[k]*(self.alpha[k] - alpha2)
                        # alpha必须都大于0，否则不更新
                        if (alpha1 < 0) | (alpha2 < 0):
                            continue
                        alpha_new = self.alpha.copy()
                        alpha_new[j] = alpha1
                        alpha_new[k] = alpha2
                        obj_new = self.compute_obj(alpha_new)
                        # 目标函数是否可观下降
                        if (obj - obj_new < self.tol):
                            continue
                        else:
                            break
                    else:
                        flag += 1
                        continue
                    out_loop = False
                    break
            b1 = (-self.E[j] - self.Y[j] * self.K[j,j] * (alpha1 - self.alpha[j]) 
                    - self.Y[k] * self.K[k,j]* (alpha2 - self.alpha[k]) + self.b)
            b2 = (-self.E[k] - self.Y[j] * self.K[j,k] * (alpha1 - self.alpha[j]) 
                    - self.Y[k] * self.K[k,k]* (alpha2 - self.alpha[k]) + self.b)
            self.b = (b1+b2)/2
            self.alpha[j] = alpha1
            self.alpha[k] = alpha2
            self.g = self.compute_g() 
            self.E = self.g - self.Y 
    def predict(self, newdata, thereshold = 1e-4):
        idx = self.alpha != 0
        K = self.compute_K(self.X[idx],newdata)
        tmp1 = self.alpha * self.Y
        tmp2 = np.dot(tmp1[idx],K) + self.b
        return np.where(tmp2>=thereshold, 1, -1)

In [32]:
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1
svm = SVM(Y = y_train,
          X = X_train,
          C = 1,
          epoch_num = 100,
          kernel = "linear")
svm.smo()
np.mean(svm.predict(X_test) == y_test)

0.935672514619883

In [36]:
svm = SVM(Y = y_train,
          X = X_train,
          C = 1,
          epoch_num = 100,
          kernel = "gaussian",
          sigma = 7.5)
svm.smo()
np.mean(svm.predict(X_test) == y_test)

0.8888888888888888

结果挺奇怪的，可能代码里有bug。。。

In [37]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

## 树模型

### 决策树

In [9]:
class Tree:
    def __init__(self, Y, X, min_node_size, max_node_depth, rule = None, threshold = 1e-4):
        # 数据存储
        self.Y = Y
        self.X = X
        self.n, self.feature_num = X.shape
        # 递归控制
        self.gini = self.compute_gini(Y)
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        # 分裂规则
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        # 节点预测
        self.threshold = 1e-4
        self.ybar = np.mean(Y)
        self.yhat = 1 if self.ybar >= (0.5+self.threshold) else 0
        # 规则记录
        self.rule = rule if rule is not None else ""
    @staticmethod
    def compute_gini(y):
        n0 = np.sum(y == 0)
        n1 = np.sum(y == 1)
        if (n1 == 0) or (n0 == 0):
            return 0
        else:
            n = len(y)
            p0 = n0/n
            p1 = n1/n
            gini_index = 1-(p0**2 + p1**2)
            return gini_index
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        for feature in range(self.feature_num):
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:,feature] < value
                ridx = self.X[:,feature] >= value
                n_left = np.sum(lidx)
                n_right = np.sum(ridx)
                if (n_left == 0) or (n_right == 0):
                    continue
                gini_left = self.compute_gini(self.Y[lidx])
                gini_right = self.compute_gini(self.Y[ridx])
                gini_gain = self.gini - (n_left * gini_left + n_right * gini_right)/self.n
                if gini_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = gini_gain
        return (best_feature,best_value)
    def fit(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n) and (self.gini > 0):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:,self.best_feature] < self.best_value
                ridx = self.X[:,self.best_feature] >= self.best_value
                left = Tree(Y = self.Y[lidx],
                            X = self.X[lidx,:],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                right = Tree(Y = self.Y[ridx],
                             X = self.X[ridx,:],
                             min_node_size = self.min_node_size,
                             max_node_depth = self.max_node_depth - 1,
                             rule = f"X{self.best_feature+1} >= {self.best_value}")
                left.fit()
                right.fit()
                self.left = left
                self.right = right
    def predict_prob(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            if (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n) and (cur_node.gini > 0):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.ybar
        return yhat
    def predict(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n) and (cur_node.gini > 0):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.yhat
        return yhat

In [10]:
tree = Tree(Y = y_train,
            X = X_train,
            min_node_size = 10,
            max_node_depth = 5)
tree.fit()
np.mean(tree.predict(X_test) == y_test)

0.935672514619883

接下来看集成模型的

### bagging

#### 随机森林

In [22]:
class Tree:
    def __init__(self, Y, X, min_node_size, max_node_depth, mtry, rule = None, threshold = 1e-4):
        # 数据存储
        self.Y = Y
        self.X = X
        self.n, self.feature_num = X.shape
        # 递归控制
        self.gini = self.compute_gini(Y)
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        # 分裂规则
        self.best_feature = None
        self.best_value = None
        self.mtry = mtry
        self.left = None
        self.right = None
        # 节点预测
        self.threshold = 1e-4
        self.ybar = np.mean(Y)
        self.yhat = 1 if self.ybar >= (0.5+self.threshold) else 0
        # 规则记录
        self.rule = rule if rule is not None else ""
    @staticmethod
    def compute_gini(y):
        n0 = np.sum(y == 0)
        n1 = np.sum(y == 1)
        if (n1 == 0) or (n0 == 0):
            return 0
        else:
            n = len(y)
            p0 = n0/n
            p1 = n1/n
            gini_index = 1-(p0**2 + p1**2)
            return gini_index
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        feature_try = np.random.choice(np.arange(self.feature_num), self.mtry, replace = False)
        for feature in feature_try:
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:,feature] < value
                ridx = self.X[:,feature] >= value
                n_left = np.sum(lidx)
                n_right = np.sum(ridx)
                if (n_left == 0) or (n_right == 0):
                    continue
                gini_left = self.compute_gini(self.Y[lidx])
                gini_right = self.compute_gini(self.Y[ridx])
                gini_gain = self.gini - (n_left * gini_left + n_right * gini_right)/self.n
                if gini_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = gini_gain
        return (best_feature,best_value)
    def fit(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n) and (self.gini > 0):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:,self.best_feature] < self.best_value
                ridx = self.X[:,self.best_feature] >= self.best_value
                left = Tree(Y = self.Y[lidx],
                            X = self.X[lidx,:],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            mtry = self.mtry,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                right = Tree(Y = self.Y[ridx],
                             X = self.X[ridx,:],
                             min_node_size = self.min_node_size,
                             max_node_depth = self.max_node_depth - 1,
                             mtry = self.mtry,
                             rule = f"X{self.best_feature+1} >= {self.best_value}")
                left.fit()
                right.fit()
                self.left = left
                self.right = right
    def predict_prob(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            if (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n) and (cur_node.gini > 0):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.ybar
        return yhat
    def predict(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n) and (cur_node.gini > 0):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.yhat
        return yhat

In [25]:
from tqdm import tqdm
class RandomForest:
    def __init__(self, Y, X, min_node_size, max_node_depth, ntree, mtry):
        self.Y = Y
        self.X = X
        self.n = X.shape[0]
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.ntree = ntree
        self.mtry = mtry
        self.tree_set = []
    def fit(self):
        for i in tqdm(range(self.ntree)):
            bootstrap_index = np.random.choice(np.arange(self.n), self.n, replace = True)
            X = self.X[bootstrap_index,]
            Y = self.Y[bootstrap_index]
            tree = Tree(Y = Y,
                        X = X,
                        min_node_size = self.min_node_size,
                        max_node_depth = self.max_node_depth,
                        mtry = self.mtry)
            tree.fit()
            self.tree_set.append(tree)
    def predict(self, newdata, threshold = 1e-4):
        n = newdata.shape[0]
        tmp = np.zeros((self.ntree,n))
        for i in range(self.ntree):
            tmp[i,] = self.tree_set[i].predict(newdata)
        vote_mean = np.mean(tmp, axis = 0)
        return np.where(vote_mean > (0.5+threshold), 1, 0)

In [33]:
np.random.seed(1118)
rf = RandomForest(Y = y_train,
                  X = X_train,
                  min_node_size = 10,
                  max_node_depth = 4,
                  ntree = 10,
                  mtry = 4)
rf.fit()
np.mean(rf.predict(X_test) == y_test)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.25it/s]


0.9824561403508771

### boosting

#### Adaboost

In [45]:
class Tree:
    def __init__(self, Y, X, min_node_size, max_node_depth, weight = None, rule = None):
        # 数据
        self.Y = Y
        self.X = X
        self.n, self.n_feature = X.shape
        # 递归控制
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        # 节点分裂
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        # 节点预测
        self.yhat = self.node_predict(Y)
        # 样本点权重
        self.weight = weight if weight is not None else np.ones(self.n)/self.n
        # 节点误差率
        self.err = self.compute_err(Y, self.yhat, self.weight)
        # 规则记录
        self.rule = rule if rule is not None else ""
    @staticmethod
    def node_predict(y, threshold = 1e-4):
        tmp = np.mean(y)
        yhat = 1 if tmp > threshold else -1
        return yhat
    @staticmethod
    def compute_err(y, yhat, weight):
        return np.dot((y != yhat).astype(int), weight)
    def split_find(self):
        best_feature = None
        best_value = None
        max_gain = 0
        for feature in range(self.n_feature):
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:,feature] < value
                ridx = self.X[:,feature] >= value
                if (np.sum(lidx) == 0) or (np.sum(ridx) == 0):
                    continue
                y_left = self.Y[lidx]
                y_right = self.Y[ridx]
                w_left = self.weight[lidx]
                w_right = self.weight[ridx]
                err_new = (self.compute_err(y_left, self.node_predict(y_left), w_left)+
                          self.compute_err(y_right, self.node_predict(y_right), w_right))
                err_gain = self.err - err_new
                if err_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = err_gain
        return (best_feature, best_value)
    def fit(self):
        if (self.min_node_size < self.n) and (self.max_node_depth > 0) and (self.err > 0):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:,self.best_feature] < self.best_value
                ridx = self.X[:,self.best_feature] >= self.best_value
                left = Tree(X = self.X[lidx,],
                            Y = self.Y[lidx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            weight = self.weight[lidx],
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                right = Tree(X = self.X[ridx,],
                             Y = self.Y[ridx],
                             min_node_size = self.min_node_size,
                             max_node_depth = self.max_node_depth - 1,
                             weight = self.weight[ridx],
                             rule = f"X{self.best_feature+1} >= {self.best_value}")
                left.fit()
                right.fit()
                self.left = left
                self.right = right
    def predict(self, newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n) and (cur_node.err > 0):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.yhat
        return yhat

In [46]:
class Adaboost:
    def __init__(self, Y, X, min_node_size, max_node_depth, nboost):
        self.Y = Y
        self.X = X
        self.n = X.shape[0]
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.nboost = nboost
        self.tree_set = []
        self.alpha = []
    @staticmethod
    def compute_err(y, yhat, weight):
        return np.dot((y != yhat).astype(int), weight)
    def fit(self):
        weight = np.ones(self.n)/self.n
        for i in tqdm(range(self.nboost)):
            tree = Tree(X = self.X,
                        Y = self.Y,
                        min_node_size = self.min_node_size,
                        max_node_depth = self.max_node_depth,
                        weight = weight)
            tree.fit()
            yhat = tree.predict(self.X)
            err = self.compute_err(self.Y, yhat, weight)
            alpha = np.log((1-err)/err)/2
            w = weight * np.exp(-alpha * self.Y * yhat)
            weight = w/np.sum(w)
            self.tree_set.append(tree)
            self.alpha.append(alpha)
    def predict(self, newdata, threshold = 1e-4):
        n = newdata.shape[0]
        cum = np.zeros(n)
        for i in range(self.nboost):
            cum += self.alpha[i] * self.tree_set[i].predict(newdata)
        return np.where(cum > threshold, 1, -1)

In [50]:
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1
adaboost = Adaboost(X = X_train,
                    Y = y_train,
                    min_node_size = 10,
                    max_node_depth = 4,
                    nboost = 10)
adaboost.fit()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.47s/it]


In [52]:
np.mean(adaboost.predict(X_test) == y_test)

0.9707602339181286

这次结果确实好看，不过训练速度有点慢。。

#### GBDT

In [10]:
# GBDT的基学习器
class Tree:
    def __init__(self, Y, X, P, min_node_size, max_node_depth, rule = None):
        # 数据存储
        self.Y = Y
        self.X = X
        self.P = P
        self.n, self.feature_num = X.shape
        # 递归控制
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.n = len(Y)
        # 节点分裂信息
        self.mse = np.var(Y)
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        # 节点预测值
        self.yhat = np.sum(Y)/np.sum(P * (1-P))
        # 字符串记录，可以用来调试
        self.rule = rule if rule is not None else ""
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        for feature in range(self.feature_num):
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:, feature] < value
                ridx = self.X[:, feature] >= value
                n_left = np.sum(lidx)
                n_right = np.sum(ridx)
                # 注意这里只是跳出本次循环
                if (n_left == 0) or (n_right == 0):
                    continue
                mse_left = np.var(self.Y[lidx])
                mse_right = np.var(self.Y[ridx])
                mse_gain = self.mse - (n_left * mse_left + n_right * mse_right) / self.n
                if mse_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = mse_gain
        return (best_feature, best_value)
    def fit(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:, self.best_feature] < self.best_value
                ridx = self.X[:, self.best_feature] >= self.best_value
                left = Tree(Y = self.Y[lidx],
                            X = self.X[lidx],
                            P = self.P[lidx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                left.fit()
                right = Tree(Y = self.Y[ridx],
                             X = self.X[ridx],
                             P = self.P[ridx],
                             min_node_size = self.min_node_size,
                             max_node_depth = self.max_node_depth - 1,
                             rule = f"X{self.best_feature+1} >= {self.best_value}")
                right.fit()
                self.left = left
                self.right = right
    def predict(self,newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.yhat
        return yhat

In [14]:
import numpy as np
import pandas as pd
from tqdm import tqdm
class GBDTclassifier:
    def __init__(self, Y, X, min_node_size, max_node_depth, nboost, lr = 0.5):
        self.Y = Y
        self.X = X
        self.n = X.shape[0]
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.nboost = nboost
        self.lr = lr
        self.log_odds = np.log(np.sum(Y)/np.sum(1-Y))
        self.tree_set = []
    @staticmethod
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    def fit(self):
        yhat = np.ones(self.n) * self.log_odds
        for i in tqdm(range(self.nboost)):
            P = self.sigmoid(yhat)
            res = self.Y - P
            tree = Tree(Y = res,
                        X = self.X,
                        P = P,
                        min_node_size = self.min_node_size,
                        max_node_depth = self.max_node_depth)
            tree.fit()
            yhat += tree.predict(self.X) * self.lr
            self.tree_set.append(tree)
    def predict_prob(self, newdata):
        p = self.log_odds.copy()
        for tree in self.tree_set:
            p += tree.predict(newdata) * self.lr
        return p
    def predict(self, newdata, threshold = 1e-4):
        p = self.predict_prob(newdata)
        return np.where(p >= (0.5+threshold), 1, 0)

In [15]:
from sklearn import datasets
X,y = datasets.load_breast_cancer(return_X_y=True)
# 划分训练集和测试集
np.random.seed(104)
index1 = np.random.choice(X.shape[0],round(X.shape[0]*0.7),replace = False)
index2 = np.array(list(set(range(X.shape[0]))-set(index1)))
X_train, X_test, y_train, y_test = X[index1,].copy(), X[index2,].copy(), y[index1].copy(), y[index2].copy()

In [20]:
gbdt = GBDTclassifier(Y = y_train,
                      X = X_train,
                      min_node_size = 10,
                      max_node_depth = 4,
                      nboost = 10,
                      lr = 0.7)
gbdt.fit()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.61s/it]


In [21]:
np.mean(gbdt.predict(X_test) == y_test)

0.9766081871345029

#### Xgboost

In [22]:
# 这次尝试一下X为ndarray
class Tree:
    def __init__(self, X, grad, hess, min_node_size, max_node_depth, mtry, lambda_ = 1, gamma = 1, 
                 eps = 0.1, rule = None):
        self.X = X
        self.n, self.feature_num = X.shape
        self.grad = grad
        self.hess = hess
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.mtry = mtry
        self.lambda_ = lambda_
        self.gamma = gamma
        self.eps = eps
        self.score = self.compute_score(self.grad, self.hess) #叶节点得分
        self.rule = rule if rule is not None else ""
        self.best_feature = None
        self.best_value = None
        self.left = None
        self.right = None
        self.leaf_weight = self.compute_weight(self.grad, self.hess)
    def compute_score(self, grad, hess):
        return -1/2 * np.sum(grad)**2 / (np.sum(hess) + self.lambda_)
    def compute_weight(self, grad, hess):
        return - np.sum(grad) / (np.sum(hess) + self.lambda_)
    def split_find(self):
        max_gain = 0
        best_feature = None
        best_value = None
        # Exact Greedy Algorithm
        feature_try = np.random.choice(np.arange(self.feature_num), self.mtry, replace = False)
        for feature in feature_try:
            grid = np.sort(np.unique(self.X[:,feature]))
            for value in grid:
                lidx = self.X[:, feature] < value
                ridx = self.X[:, feature] >= value
                lscore = self.compute_score(self.grad[lidx], self.hess[lidx])
                rscore = self.compute_score(self.grad[ridx], self.hess[ridx])
                score_gain = (self.score - lscore - rscore)/2 - self.gamma
                if score_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    max_gain = score_gain
        return (best_feature, best_value)
    def grow_tree(self):
        if (self.max_node_depth > 0) and (self.min_node_size < self.n):
            self.best_feature,self.best_value = self.split_find()
            if self.best_feature is not None:
                lidx = self.X[:, self.best_feature] < self.best_value
                ridx = self.X[:, self.best_feature] >= self.best_value
                left = Tree(X = self.X[lidx],
                            grad = self.grad[lidx],
                            hess = self.hess[lidx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            mtry = self.mtry,
                            lambda_ = self.lambda_,
                            gamma = self.gamma,
                            eps = self.eps,
                            rule = f"X{self.best_feature+1} < {self.best_value}")
                left.grow_tree()
                right = Tree(X = self.X[ridx],
                            grad = self.grad[ridx],
                            hess = self.hess[ridx],
                            min_node_size = self.min_node_size,
                            max_node_depth = self.max_node_depth - 1,
                            mtry = self.mtry,
                            lambda_ = self.lambda_,
                            gamma = self.gamma,
                            eps = self.eps,
                            rule = f"X{self.best_feature+1} >= {self.best_value}")
                right.grow_tree()
                self.left = left
                self.right = right
    def predict(self, newdata):
        n = newdata.shape[0]
        yhat = np.zeros(n)
        for i in range(n):
            cur_node = self
            while (cur_node.max_node_depth > 0) and (cur_node.min_node_size < cur_node.n):
                if cur_node.best_feature is None:
                    break
                if newdata[i,cur_node.best_feature] < cur_node.best_value:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            yhat[i] = cur_node.leaf_weight
        return yhat

In [23]:
class XgboostClassifier:
    def __init__(self, X, Y, min_node_size, max_node_depth, mtry, nboost, lambda_ = 1, gamma = 1, eps = 0.1, lr = 1):
        self.X = X
        self.n,self.p = X.shape
        self.Y = Y
        self.min_node_size = min_node_size
        self.max_node_depth = max_node_depth
        self.mtry = mtry
        self.nboost = nboost
        self.lambda_ = lambda_
        self.gamma = gamma
        self.eps = eps
        self.lr = lr
        self.tree_set = []
        self.base_pred = np.ones_like(Y) * self.log_odds(Y)
    @staticmethod
    def sigmoid(x):
        return 1 / (1+np.exp(-x))
    @staticmethod
    def log_odds(y):
        odds = np.sum(y == 1)/np.sum(y == 0)
        return np.log(odds)
    
    def fit(self):
        for i in tqdm(range(self.nboost)):
            p = self.sigmoid(self.base_pred)
            grad = p - self.Y
            hess = p * (1 - p)
            tree = Tree(X = self.X,
                        grad = grad,
                        hess = hess,
                        min_node_size = self.min_node_size,
                        max_node_depth = self.max_node_depth,
                        mtry = self.mtry,
                        lambda_ = self.lambda_,
                        gamma = self.gamma,
                        eps = self.eps)
            tree.grow_tree()
            self.base_pred += self.lr * tree.predict(self.X)
            self.tree_set.append(tree)
    def predict_prob(self,newdata):
        n = newdata.shape[0]
        yhat = np.ones(n) * self.log_odds(self.Y)
        for tree in self.tree_set:
            yhat += self.lr * tree.predict(newdata)
        return self.sigmoid(yhat)
    def predict(self, newdata, threshold = (0.5+1e-4)):
        p = self.predict_prob(newdata)
        pred = np.where(p >= threshold, 1, 0)
        return pred

In [25]:
np.random.seed(1118)
xgbooster = XgboostClassifier(X = X_train,
                              Y = y_train,
                              min_node_size = 10,
                              max_node_depth = 4,
                              mtry = 5,
                              nboost = 30,
                              lr = 0.7,
                              gamma = 2)
xgbooster.fit()
np.mean(xgbooster.predict(X_test) == y_test)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:04<00:00,  6.84it/s]


0.9883040935672515